# Lean 9 : LeanDojo - ML/LLM Theorem Proving

## Introduction

**LeanDojo** est une bibliothèque Python qui permet l'interaction programmatique avec Lean 4 pour le machine learning et la démonstration automatique de théorèmes.

### Capacités de LeanDojo

| Fonctionnalité | Description |
|----------------|-------------|
| **Tracing** | Analyse et extraction de métadonnées d'un dépôt Lean |
| **Theorem Extraction** | Liste tous les théorèmes avec leurs types et preuves |
| **Dojo Environment** | REPL interactif pour exécuter des tactiques |
| **Tactic State** | Accès à l'état de preuve (goals, hypothèses) |

### Architecture

```
LeanDojo
├── LeanGitRepo      # Référence à un dépôt Git Lean
├── trace()          # Analyse le dépôt et extrait les métadonnées
├── TracedRepo       # Dépôt analysé avec accès aux théorèmes
├── Theorem          # Représentation d'un théorème
└── Dojo             # Environnement interactif pour prouver
    ├── run_tac()    # Exécute une tactique
    └── TacticState  # État après chaque tactique
```

### Prérequis

- Python 3.10-3.12 (pas 3.13+)
- GitHub token pour l'API (rate limiting)
- ~5 Go d'espace disque pour le cache

**Durée estimée** : 45 minutes

## 1. Installation et Configuration

In [ ]:
# Vérification de la version Python
import sys
print(f"Python: {sys.version}")

# LeanDojo nécessite Python < 3.13
if sys.version_info >= (3, 13):
    print("WARNING: LeanDojo requires Python < 3.13")
    print("Use: conda activate mcp-jupyter-py310")

In [ ]:
# Installation de LeanDojo (si nécessaire)
# !pip install lean-dojo

In [ ]:
# Configuration du token GitHub
import os
from pathlib import Path

def load_github_token():
    """Charge le token GitHub depuis .env ou gh CLI."""
    # 1. Déjà dans l'environnement
    if "GITHUB_ACCESS_TOKEN" in os.environ:
        return True
    
    # 2. Fichier .env local
    env_path = Path(".") / ".env"
    if env_path.exists():
        with open(env_path) as f:
            for line in f:
                if line.strip().startswith("GITHUB_ACCESS_TOKEN="):
                    os.environ["GITHUB_ACCESS_TOKEN"] = line.split("=", 1)[1].strip()
                    return True
                elif line.strip().startswith("GITHUB_TOKEN="):
                    os.environ["GITHUB_ACCESS_TOKEN"] = line.split("=", 1)[1].strip()
                    return True
    
    # 3. gh CLI
    try:
        import subprocess
        result = subprocess.run(["gh", "auth", "token"], capture_output=True, text=True)
        if result.returncode == 0:
            os.environ["GITHUB_ACCESS_TOKEN"] = result.stdout.strip()
            return True
    except:
        pass
    
    return False

if load_github_token():
    token = os.environ["GITHUB_ACCESS_TOKEN"]
    print(f"GitHub token: OK ({token[:10]}...)")
else:
    print("WARNING: No GitHub token found")
    print("Set GITHUB_ACCESS_TOKEN in .env or run 'gh auth login'")

## 2. Imports et Vérification

In [ ]:
# Import des modules LeanDojo
try:
    import lean_dojo
    print(f"LeanDojo version: {lean_dojo.__version__}")
    
    from lean_dojo import (
        LeanGitRepo,      # Référence à un dépôt Lean
        trace,            # Fonction de tracing
        is_available_in_cache,  # Vérifie le cache
        Dojo,             # Environnement interactif
        Theorem,          # Représentation d'un théorème
        TacticState,      # État de preuve
    )
    print("All imports: OK")
except ImportError as e:
    print(f"ERROR: {e}")
    print("Run: pip install lean-dojo")

## 3. Création d'une Référence de Dépôt

`LeanGitRepo` représente un dépôt Git Lean. Il ne télécharge rien - c'est juste une référence.

**Important** : LeanDojo 4.x nécessite des dépôts Lean 4 avec un fichier `lean-toolchain`.

In [ ]:
# Exemple officiel LeanDojo (petit, rapide)
EXAMPLE_REPO = {
    "url": "https://github.com/yangky11/lean4-example",
    "commit": "7761283d0aed994cd1c7e893786212d2a01d159e",
}

# Création de la référence
repo = LeanGitRepo(EXAMPLE_REPO["url"], EXAMPLE_REPO["commit"])

print(f"Repository: {repo.url}")
print(f"Commit: {repo.commit[:12]}")
print(f"Exists on GitHub: {repo.exists()}")
print(f"In local cache: {is_available_in_cache(repo)}")

## 4. Tracing d'un Dépôt

Le **tracing** est l'étape clé de LeanDojo. Il :

1. Clone le dépôt Git
2. Télécharge les dépendances (Mathlib4, etc.)
3. Compile le projet avec instrumentation
4. Extrait les métadonnées (théorèmes, tactiques, états)

**Note** : Le premier tracing peut prendre plusieurs minutes et télécharger ~1 Go.

In [ ]:
import time

print("Tracing repository...")
print("(First run downloads dependencies and may take several minutes)")

start = time.time()
traced_repo = trace(repo)
elapsed = time.time() - start

print(f"\nTrace complete in {elapsed:.1f}s")
print(f"Traced repo path: {traced_repo.repo_path}")

## 5. Extraction des Théorèmes

Une fois le dépôt tracé, on peut extraire tous les théorèmes.

In [ ]:
# Extraction des théorèmes
theorems = list(traced_repo.get_theorems())
print(f"Found {len(theorems)} theorems")

# Affichage des premiers théorèmes
print("\nSample theorems:")
for i, thm in enumerate(theorems[:10]):
    print(f"  [{i+1}] {thm.full_name}")
    print(f"      File: {thm.file_path}")

In [ ]:
# Détails d'un théorème
if theorems:
    thm = theorems[0]
    print(f"Theorem: {thm.full_name}")
    print(f"File: {thm.file_path}")
    print(f"Module: {thm.file_path.stem}")
    
    # Le théorème contient aussi le code source
    if hasattr(thm, 'code'):
        print(f"\nCode:\n{thm.code}")

## 6. Environnement Dojo - Preuves Interactives

**Dojo** est l'environnement interactif de LeanDojo. Il permet :

- D'ouvrir un théorème pour le prouver
- D'exécuter des tactiques une par une
- D'observer l'état de preuve après chaque tactique

C'est l'interface idéale pour les LLMs qui génèrent des preuves.

In [ ]:
# Ouvrir un Dojo pour un théorème
if theorems:
    thm = theorems[0]
    print(f"Opening Dojo for: {thm.full_name}")
    
    with Dojo(thm) as (dojo, init_state):
        print(f"\nInitial state:")
        print(f"  Goals: {len(init_state.goals)}")
        
        if init_state.goals:
            print(f"  Goal 0: {init_state.goals[0]}")
        
        # Pretty-print de l'état
        print(f"\n  Pretty-printed state:")
        print(f"  {init_state.pp}")

In [ ]:
# Exécution de tactiques
if theorems:
    thm = theorems[0]
    
    with Dojo(thm) as (dojo, init_state):
        print(f"Theorem: {thm.full_name}")
        print(f"Initial goals: {len(init_state.goals)}")
        
        # Essayer plusieurs tactiques
        tactics_to_try = ["intro", "intros", "rfl", "simp", "trivial"]
        
        current_state = init_state
        for tactic in tactics_to_try:
            if current_state is None or not current_state.goals:
                print(f"\nProof complete!")
                break
            
            result = dojo.run_tac(current_state, tactic)
            if result is not None:
                print(f"\n'{tactic}' succeeded:")
                print(f"  Goals remaining: {len(result.goals)}")
                if result.goals:
                    print(f"  New goal: {result.goals[0]}")
                current_state = result
            else:
                print(f"'{tactic}' failed (expected)")

## 7. Utilisation avec lean_runner.py

Le module `lean_runner.py` fournit une interface simplifiée pour LeanDojo.

In [ ]:
# Import de lean_runner
import sys
sys.path.insert(0, '.')

from lean_runner import LeanRunner

# Créer un runner avec backend LeanDojo
runner = LeanRunner(backend='leandojo')
print(f"Backend: {runner.backend}")

In [ ]:
# Tracer un dépôt via lean_runner
traced = runner.trace_repo(
    "https://github.com/yangky11/lean4-example",
    "7761283d0aed994cd1c7e893786212d2a01d159e"
)
print(f"Traced: {traced}")

In [ ]:
# Extraire les théorèmes
thms = runner.get_theorems()
print(f"Found {len(thms)} theorems")

for t in thms[:5]:
    print(f"  - {t.full_name}")

In [ ]:
# Tenter de prouver avec une séquence de tactiques
if thms:
    result = runner.prove_with_tactics(
        thms[0],
        ["intro", "rfl", "simp"]
    )
    
    print(f"Proof attempt:")
    print(f"  Success: {result['success']}")
    print(f"  Steps: {len(result['steps'])}")
    for step in result['steps']:
        print(f"    {step['tactic']}: {'OK' if step['success'] else 'FAIL'}")

## 8. Dépôts Avancés : Mathlib et formal-conjectures

LeanDojo peut tracer des dépôts plus complexes comme :

- **Mathlib4** : La bibliothèque mathématique principale de Lean 4
- **formal-conjectures** : Conjectures formalisées par Google DeepMind

**Note** : Ces dépôts sont grands et le premier tracing peut prendre 30+ minutes.

In [ ]:
# Dépôts disponibles pour tests
ADVANCED_REPOS = {
    "formal-conjectures": {
        "url": "https://github.com/google-deepmind/formal-conjectures",
        "commit": "ce0a081ab74d625948c44da6022992e1f9db070a",
        "lean_version": "4.22.0",
        "description": "Google DeepMind formalized conjectures",
    },
    # Mathlib4 est très grand, à utiliser avec précaution
    # "mathlib4": {
    #     "url": "https://github.com/leanprover-community/mathlib4",
    #     "commit": "v4.15.0",
    #     "lean_version": "4.15.0",
    # },
}

print("Available advanced repos:")
for name, info in ADVANCED_REPOS.items():
    print(f"  {name}: {info['description']}")

In [ ]:
# Exemple avec formal-conjectures (décommentez pour exécuter)
# ATTENTION: Télécharge Mathlib4 (~1 Go) au premier lancement

# deepmind_repo = LeanGitRepo(
#     ADVANCED_REPOS["formal-conjectures"]["url"],
#     ADVANCED_REPOS["formal-conjectures"]["commit"]
# )
# 
# print(f"Tracing formal-conjectures...")
# traced_deepmind = trace(deepmind_repo)
# 
# theorems = list(traced_deepmind.get_theorems())
# print(f"Found {len(theorems)} theorems/conjectures")

## 9. Intégration avec LLMs

LeanDojo est conçu pour l'intégration avec des LLMs. Le workflow typique :

1. **Extraire** les théorèmes d'un dépôt
2. **Ouvrir** un Dojo pour un théorème
3. **Envoyer** l'état au LLM
4. **Recevoir** une tactique du LLM
5. **Exécuter** la tactique
6. **Répéter** jusqu'à ce que la preuve soit complète ou échoue

Voir le notebook **Lean-7-LLM-Integration.ipynb** pour des exemples concrets.

In [ ]:
# Exemple de prompt pour un LLM
def format_state_for_llm(state):
    """Formate l'état de preuve pour envoi à un LLM."""
    prompt = "Given the following Lean 4 proof state:\n\n"
    prompt += f"```\n{state.pp}\n```\n\n"
    prompt += "Suggest the next tactic to apply. "
    prompt += "Respond with only the tactic, no explanation."
    return prompt

# Exemple d'utilisation (sans appel LLM réel)
if theorems:
    thm = theorems[0]
    with Dojo(thm) as (dojo, init_state):
        prompt = format_state_for_llm(init_state)
        print("Prompt for LLM:")
        print(prompt)

## 10. Bonnes Pratiques et Limitations

### Bonnes pratiques

1. **Utilisez le cache** : Le tracing est lent, le cache accélère les exécutions suivantes
2. **Commit spécifique** : Utilisez toujours un hash de commit, pas une branche
3. **GitHub token** : Évitez le rate limiting avec un token personnel
4. **Windows** : Utilisez `freeze_support()` pour le multiprocessing

### Limitations

| Limitation | Description |
|------------|-------------|
| Lean 4 only | LeanDojo 4.x ne supporte pas Lean 3 |
| lean-toolchain | Le dépôt doit avoir ce fichier |
| Python < 3.13 | Incompatibilité avec Python 3.13+ |
| Espace disque | Cache volumineux (~5 Go pour Mathlib) |

### Références

- [LeanDojo Documentation](https://leandojo.readthedocs.io/)
- [LeanDojo Paper](https://arxiv.org/abs/2306.15626) (NeurIPS 2023)
- [lean4-example Repository](https://github.com/yangky11/lean4-example)

## Résumé

Dans ce notebook, nous avons couvert :

1. **Installation** et configuration de LeanDojo
2. **LeanGitRepo** pour référencer un dépôt Lean
3. **Tracing** pour analyser et extraire les métadonnées
4. **Extraction** des théorèmes
5. **Dojo** pour les preuves interactives
6. **lean_runner.py** pour une interface simplifiée
7. **Intégration LLM** pour la démonstration automatique

### Prochaines étapes

- **Lean-7-LLM-Integration.ipynb** : Patterns d'intégration LLM-Lean
- **Lean-8-Agentic-Proving.ipynb** : Agents autonomes pour la preuve de théorèmes